In [1]:
import torch
bart = torch.hub.load('pytorch/fairseq', 'bart.large')
bart.eval()

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_main
2023-11-29 01:03:04.683833: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-29 01:03:04.725285: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-29 01:03:05.762556: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


running build_ext


/opt/conda/lib/python3.10/site-packages/torch/utils/cpp_extension.py:502: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
tokens = bart.encode('Hello the fucking world!')
# assert tokens.tolist() == [0, 31414, 232, 328, 2]
bart.decode(tokens)  # 'Hello world!'

In [ ]:
# Extract the last layer's features
last_layer_features = bart.extract_features(tokens)
# assert last_layer_features.size() == torch.Size([1, 5, 1024])
print(last_layer_features.size())

# Extract all layer's features from decoder (layer 0 is the embedding layer)
all_layers = bart.extract_features(tokens, return_all_hiddens=True)
print(all_layers)
assert len(all_layers) == 13
assert torch.all(all_layers[-1] == last_layer_features)

In [ ]:
last_layer_features.shape

In [ ]:
bart.fill_mask(['The cat <mask> on the <mask>.'], topk=3, beam=10)


In [ ]:
!pip install fairseq